<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 803ms/step - loss: 0.6564 - accuracy: 0.6022 - val_loss: 0.6454 - val_accuracy: 0.5902
Epoch 2/10
26/26 [==============================] - 20s 757ms/step - loss: 0.6005 - accuracy: 0.6849 - val_loss: 0.7183 - val_accuracy: 0.4537
Epoch 3/10
26/26 [==============================] - 20s 762ms/step - loss: 0.5492 - accuracy: 0.7591 - val_loss: 0.4894 - val_accuracy: 0.8195
Epoch 4/10
26/26 [==============================] - 20s 754ms/step - loss: 0.4829 - accuracy: 0.8041 - val_loss: 0.4218 - val_accuracy: 0.8780
Epoch 5/10
26/26 [==============================] - 20s 757ms/step - loss: 0.4096 - accuracy: 0.8516 - val_loss: 0.4511 - val_accuracy: 0.8000
Epoch 6/10
26/26 [==============================] - 19s 747ms/step - loss: 0.3320 - accuracy: 0.8796 - val_loss: 0.2873 - val_accuracy: 0.9024
Epoch 7/10
26/26 [==============================] - 19s 735ms/step - loss: 0.2939 - accuracy: 0.9063 - val_loss: 0.2335 - val_accuracy: 0.9220

In [34]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1176), started 0:37:49 ago. (Use '!kill 1176' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.61.h5
26/26 - 20s - loss: 0.6615 - accuracy: 0.6034 - val_loss: 0.6078 - val_accuracy: 0.7024
Epoch 2/5

Epoch 00002: saving model to weights.02-0.55.h5
26/26 - 20s - loss: 0.5796 - accuracy: 0.7019 - val_loss: 0.5514 - val_accuracy: 0.7951
Epoch 3/5

Epoch 00003: saving model to weights.03-0.49.h5
26/26 - 20s - loss: 0.5069 - accuracy: 0.7543 - val_loss: 0.4879 - val_accuracy: 0.8049
Epoch 4/5

Epoch 00004: saving model to weights.04-0.45.h5
26/26 - 20s - loss: 0.4270 - accuracy: 0.8114 - val_loss: 0.4488 - val_accuracy: 0.8000
Epoch 5/5

Epoch 00005: saving model to weights.05-0.37.h5
26/26 - 20s - loss: 0.3975 - accuracy: 0.8394 - val_loss: 0.3734 - val_accuracy: 0.8732


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6802 - accuracy: 0.5657 - val_loss: 0.6793 - val_accuracy: 0.4390


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6685 - accuracy: 0.6046 - val_loss: 0.6603 - val_accuracy: 0.5756
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 22s - loss: 0.6155 - accuracy: 0.6886 - val_loss: 0.6189 - val_accuracy: 0.6341


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6716 - accuracy: 0.5900 - val_loss: 0.6741 - val_accuracy: 0.4390
Epoch 2/50
26/26 - 19s - loss: 0.6364 - accuracy: 0.6825 - val_loss: 0.6349 - val_accuracy: 0.6000
Epoch 3/50
26/26 - 19s - loss: 0.5903 - accuracy: 0.7202 - val_loss: 0.5675 - val_accuracy: 0.7268
Epoch 4/50
26/26 - 20s - loss: 0.5432 - accuracy: 0.7433 - val_loss: 0.4974 - val_accuracy: 0.7756
Epoch 5/50
26/26 - 19s - loss: 0.4871 - accuracy: 0.7871 - val_loss: 0.4467 - val_accuracy: 0.8341
Epoch 6/50
26/26 - 20s - loss: 0.4080 - accuracy: 0.8273 - val_loss: 0.5492 - val_accuracy: 0.6878
Epoch 7/50
26/26 - 21s - loss: 0.3871 - accuracy: 0.8273 - val_loss: 0.3663 - val_accuracy: 0.8780
Epoch 8/50
26/26 - 21s - loss: 0.3414 - accuracy: 0.8577 - val_loss: 0.3433 - val_accuracy: 0.8829
Epoch 9/50
26/26 - 22s - loss: 0.2942 - accuracy: 0.8917 - val_loss: 0.2441 - val_accuracy: 0.8976
Epoch 10/50
26/26 - 20s - loss: 0.2367 - accuracy: 0.9209 - val_loss: 0.1862 - val_accuracy: 0.9610
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 22s 835ms/step - loss: 0.6636 - accuracy: 0.5730 - val_loss: 0.6344 - val_accuracy: 0.7854
Epoch 2/5
26/26 [==============================] - 20s 766ms/step - loss: 0.6091 - accuracy: 0.7141 - val_loss: 0.6383 - val_accuracy: 0.5366
Epoch 3/5
26/26 [==============================] - 21s 812ms/step - loss: 0.5518 - accuracy: 0.7470 - val_loss: 0.5356 - val_accuracy: 0.7610
Epoch 4/5
26/26 [==============================] - 21s 792ms/step - loss: 0.4827 - accuracy: 0.7798 - val_loss: 0.4765 - val_accuracy: 0.8195
Epoch 5/5
26/26 [==============================] - 20s 761ms/step - loss: 0.4460 - accuracy: 0.7956 - val_loss: 0.4221 - val_accuracy: 0.8585


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.572993,0.664050,0.785366,0.634411
1,1,0.714112,0.608289,0.536585,0.638254
2,2,0.746959,0.552116,0.760976,0.535631
3,3,0.779805,0.483555,0.819512,0.476476
4,4,0.795620,0.444940,0.858537,0.422150


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 33s 1s/step - loss: 0.6798 - accuracy: 0.5693 - val_loss: 0.6758 - val_accuracy: 0.5561

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 792ms/step - loss: 0.6598 - accuracy: 0.6277 - val_loss: 0.6667 - val_accuracy: 0.5707

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 797ms/step - loss: 0.6490 - accuracy: 0.6338 - val_loss: 0.6603 - val_accuracy: 0.6000

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 804ms/step - loss: 0.6422 - accuracy: 0.6691 - val_loss: 0.6599 - val_accuracy: 0.5756

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 22s 846ms/step - loss: 0.6400

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 24s 914ms/step - loss: 0.6759 - accuracy: 0.5693 - val_loss: 0.6441 - val_accuracy: 0.5805
Epoch 2/50
26/26 [==============================] - 21s 803ms/step - loss: 0.6077 - accuracy: 0.6995 - val_loss: 0.5641 - val_accuracy: 0.8488
Epoch 3/50
26/26 [==============================] - 22s 849ms/step - loss: 0.5698 - accuracy: 0.7324 - val_loss: 0.5521 - val_accuracy: 0.6439
Epoch 4/50
26/26 [==============================] - 21s 819ms/step - loss: 0.4933 - accuracy: 0.7822 - val_loss: 0.4278 - val_accuracy: 0.8634
Epoch 5/50
26/26 [==============================] - 22s 839ms/step - loss: 0.4296 - accuracy: 0.8151 - val_loss: 0.3894 - val_accuracy: 0.8244
Epoch 6/50
26/26 [==============================] - 21s 812ms/step - loss: 0.3912 - accuracy: 0.8309 - val_loss: 0.2973 - val_accuracy: 0.9268
Epoch 7/50
26/26 [==============================] - 21s 804ms/step - loss: 0.2912 - accuracy: 0.9088 - val_loss: 0.2208 - val_accuracy: 0.9415

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4390), started 0:16:53 ago. (Use '!kill 4390' to kill it.)